# Presentation Notebook

This notebook is meant to be used as the main "put it all together" notebook which we can use to present the models we have worked on.

In [1]:
SHOW_INTERMEDIATE_DATA = True # set this option if you want to display the data at each step, not just the final result.

### Setup: import libraries and files

In [2]:
import pandas as pd
import numpy as np
from HybridModel.test2 import heston_option_price as heston
from MonteCarlo.monte_carlo import montecarlo as mc
from HybridModel.hybrid_model import hybrid_model as hm
from BlackScholes.black_scholes import optionPrice as bs

In [3]:
AMD_2019 = pd.read_csv("Data/AMD/AMD_2019.csv")
AMD_2020 = pd.read_csv("Data/AMD/AMD_2020.csv")
AMD_2021 = pd.read_csv("Data/AMD/AMD_2021.csv")
AMD_2022 = pd.read_csv("Data/AMD/AMD_2022.csv")
AMD_2023 = pd.read_csv("Data/AMD/AMD_2023.csv")
AMD_5_yrs = pd.concat([AMD_2019, AMD_2020, AMD_2021, AMD_2022, AMD_2023]).reset_index()
AMD_3_yrs = pd.concat([AMD_2021, AMD_2022, AMD_2023]).reset_index() 

df = AMD_3_yrs # replace this with any of the defined dataframes above to use their data. (i.e. you could replace this with other stock data) We are just using AMD.
# df = df[:209]

### Calculate Volatility Off Historic Stock Prices

In [4]:
if SHOW_INTERMEDIATE_DATA:
    display(df[["date", "open", "close", "high", "low"]])

,date,open,close,high,low
0,2021-01-04,92.13,92.28,96.06,90.91
1,2021-01-05,92.13,92.77,93.21,91.41
2,2021-01-06,91.61,90.33,92.28,89.46
3,2021-01-07,91.36,95.09,95.51,91.20
4,2021-01-08,95.99,94.57,96.40,93.27
...,...,...,...,...,...
748,2023-12-22,140.44,139.62,140.68,138.31
749,2023-12-26,139.99,143.44,143.85,139.92
750,2023-12-27,144.74,146.13,146.25,143.17
751,2023-12-28,146.76,148.75,150.41,145.95


In [5]:
# This is a constant which will "Anualize" the volatility. there are around 252 trading days in a given year. 
#This is given by the space of a price interval, multiplied by how many intervals fit into the timespan we are converting the volatility to. 
#In this case interval = 1 day. 252 days fit into a year, which is what we want to convert the volatility to.
N = np.sqrt(252) 

#Now we need to convert from the prices to the returns, day to day.
percent_returns = df[["open","close", "high", "low"]].pct_change(1)
volatility_table = percent_returns.std() * N

if SHOW_INTERMEDIATE_DATA:
    display(percent_returns)
    display(volatility_table)

#This step will set the volatility we will use for later calculations, not sure which is best. Just made this as an arbitrary choice to have something.
volatility = volatility_table['open']
print(f"Calculated volatility: {volatility}")

drift_table = percent_returns.mean() * 252
drift = drift_table['open']
print(f"Calculated drift: {drift}")

,open,close,high,low
0,NaN,NaN,NaN,NaN
1,0.000000,0.005310,-0.029669,0.005500
2,-0.005644,-0.026302,-0.009977,-0.021332
3,-0.002729,0.052696,0.035002,0.019450
4,0.050679,-0.005469,0.009318,0.022697
...,...,...,...,...
748,0.017313,-0.001787,0.005001,0.006770
749,-0.003204,0.027360,0.022533,0.011641
750,0.033931,0.018753,0.016684,0.023228
751,0.013956,0.017929,0.028444,0.019417


open     0.512029
close    0.509711
high     0.451890
low      0.452472
dtype: float64

Calculated volatility: 0.5120293039206385
Calculated drift: 0.29289194340328806


### Define the other option parameters:

In [6]:
# This is the current risk free rate of the market. I took mine from https://ycharts.com/indicators/10_year_treasury_rate (the end of 2023). 
# Make sure to change this depending on the time you want to apply the model at.
RISK_FREE_RATE = 0.0523

# Next we have all of the specifics for the option we would like to price:

# The strike price of the option, (in USD)
STRIKE_PRICE = 220

# The current price of the stock we would like to model.
CURRENT_STOCK_PRICE = 211.38

# Time until option expiry (make sure to stay consistent in units with volatility and risk_free_rate. In our case , we use days.
TIME_TILL_EXPIRY = 25

### Calculate the option price(s):

In [7]:
# This will use the montecarlo and blackscholes libraries we have built to return the expected stock price of an option with the parameters defined above.
# Will assume that it is a call option.
print(f"""
Estimated option prices:
Monte Carlo: ${mc(138.56, STRIKE_PRICE, TIME_TILL_EXPIRY/252, RISK_FREE_RATE, volatility, drift, timeSteps=1000, simulations=100000) + STRIKE_PRICE}
Black Scholes: ${bs(CURRENT_STOCK_PRICE, STRIKE_PRICE, RISK_FREE_RATE, volatility, TIME_TILL_EXPIRY) - STRIKE_PRICE}
""")


Estimated option prices:
Monte Carlo: $220.0340584654966
Black Scholes: $-29.50653058521837



## Using our fancier Monte Carlo:(broken, probably shouldn't use)

In [8]:
sample_returns = percent_returns["close"].to_numpy()[1:] # 1st number will alays be NaN

In [9]:
# hm(CURRENT_STOCK_PRICE, RISK_FREE_RATE,41 ,sample_returns, timeSteps=5000, simulations=10000)

In [10]:
# hm(138.56, RISK_FREE_RATE, 41,sample_returns, timeSteps = 1000, simulations=50000)


## Heston Model

Find correlation coefficient between asset return and volatility:

In [11]:
historical_returns = pd.concat([AMD_2019, AMD_2020]).reset_index()
historical_returns = historical_returns["close"].pct_change(1)[1:].to_numpy()

walking_volatility = np.vectorize(lambda n: np.concatenate([ historical_returns, sample_returns[:n] ],axis=0).std() * N)
sample_volatilities = np.fromfunction(walking_volatility, sample_returns.shape, dtype=int)
corr_coef = np.corrcoef(sample_volatilities, sample_returns)[0,1]

walking_vol_no_hist = np.vectorize(lambda n: sample_returns[:n+1].std() * N)
sample_no_hist_volatilities = np.fromfunction(walking_vol_no_hist, sample_returns.shape, dtype=int)
no_hist_corr = np.corrcoef(sample_no_hist_volatilities, sample_returns)[0,1]

print(f"Correlation coefficient: {corr_coef}")
print(f"With no \"historic\" data: {no_hist_corr}")
# sample_volatilities = #

Correlation coefficient: -0.02737568831603985
With no "historic" data: -0.0010054420354741267


Find volatility of volatility: 

In [12]:
print(f"With historic data {sample_volatilities.std()}")
print(f"Without: {sample_no_hist_volatilities.std()}")

With historic data 0.01021045592591265
Without: 0.05754860949002667


Finding the mean reversion rate:

In [23]:
import statsmodels as sm
model = sm.tsa.ar_model.AutoReg(sample_returns, 1)
model_fit = model.fit()
reversion_rate = model_fit.params[1]
reversion_rate

-0.015445727286944152

### Running the model

In [24]:
heston(138.56, 41/365)

138.39242257974047